In [1]:
import os

import numpy as np
import pandas
import matplotlib.pyplot as plt
import h5py
import astropy.constants as const
import astropy.units as u
import astropy.table
import ChiantiPy.tools.util as ch_util
import ChiantiPy.tools.io as ch_io
import ChiantiPy.core as ch

%matplotlib inline

 found PyQt4 widgets
 using CLI for selections
 reading chiantirc file


In [17]:
ch_dbase = os.environ['XUVTOP']

In [128]:
os.environ['FOOBAR']

KeyError: 'FOOBAR'

# Parse Raw CHIANTI Data
In this notebook, we'll develop some tools for parsing the raw ASCII data in the CHIANTI atomic database and transforming it into pandas dataframes. It is important to include a way to very easily parse the actual raw data in the ChiantiPy package.

Then, the next step is to transform these dataframes into one large HDF5 file. ChiantiPy will then use this file like a database to stream the needed atomic data from. This provides a sleeker interface to the data itself and can be much more efficient than the current system.

## Notes on Database Structure
Other directories/files that are **not** ions:
* __abundance__
* ancillary_data
* continuum
* dem
* __ioneq__
* __ip__
* masterlist
* VERSION (file)

The items in bold can be parsed appropriately and attached to the ion data objects.

The filetypes for each ion are:
* ~~**.elvlc**~~
* ~~**.wgfa**~~
* ~~.scups~~
* ~~.psplups~~
* ~~.rrparams~~
* ~~.trparams~~
* .diparams
* .drparams
* ~~**.easplom**~~
* ~~**.easplups**~~
* ~~**.fblvl**~~
* ~~.cilvl~~
* ~~.reclvl~~

The items in bold have a basic row-column structure and can be easily read using just a list of column names and datatypes.

Those not bolded will be a bit more complicated.

One possible idea would be to force everything into the basic row-column format. This could be done by providing preprocessors for specific filetypes and then a general filereader. This would help the code be less verbose and more maintable.

For those entries which are arrays, the preprocesor could take those rows, turn them into comma separated lists, make them an item in the row (i.e. a single entry) and then give them a custom datatype that can later be used to just convert this into a Numpy array.

Goal of the preprocessor should be to force every distinct entry into a single space-delimited row where each entry is unique.

## Read and Parse Raw ASCII Data
Each CHIANTI filetype seems to have a completely different layout. We do not want the user to have to worry about this. We've implemented a basic factory pattern that creates a different parser based on the input filetype.

In [119]:
class ParserFactory(type):
    def __call__(cls,*args,**kwargs):
        filetype = args[0].split('.')[-1]
        if filetype == 'elvlc':
            return ElvlcParser(*args,**kwargs)
        elif filetype == 'wgfa':
            return WgfaParser(*args,**kwargs)
        elif filetype == 'fblvl':
            return FblvlParser(*args,**kwargs)
        elif filetype == 'scups':
            return ScupsParser(*args,**kwargs)
        elif filetype == 'easplom':
            return EasplomParser(*args,**kwargs)
        elif filetype == 'easplups':
            return EasplupsParser(*args,**kwargs)
        elif filetype == 'psplups':
            return PsplupsParser(*args,**kwargs)
        elif filetype == 'cilvl':
            return CilvlParser(*args,**kwargs)
        elif filetype == 'reclvl':
            return ReclvlParser(*args,**kwargs)
        elif filetype == 'rrparams':
            return RrparamsParser(*args,**kwargs)
        elif filetype == 'trparams':
            return TrparamsParser(*args,**kwargs)
        elif filetype == 'drparams':
            return DrparamsParser(*args,**kwargs)
        elif filetype == 'diparams':
            return DiparamsParser(*args,**kwargs)
        else:
            return type.__call__(cls,*args,**kwargs)

        
class GenericParser(object):
    dtypes = []
    units = []
    headings = []
    
    def __init__(self,ion_filename):
        self.ion_filename = ion_filename
        self.filetype = self.ion_filename.split('.')[-1]
        self.ion_name = self.ion_filename.split('.')[0]
        self.element = self.ion_name.split('_')[0]
        
    def parse(self):
        """
        Generate Astropy QTable from a CHIANTI ion file 
        """
        full_path = os.path.join(ch_dbase,self.element,self.ion_name,self.ion_filename)
        if not os.path.isfile(full_path):
            return None
        with open(full_path,'r') as f:
            lines = f.readlines()
        table = []
        for i,line in enumerate(lines):
            line = list(filter(None,line.strip().split('  ')))
            if line[0] == '-1':
                comment = ''.join(lines[i+1:len(lines)])
                break
            else:
                self.preprocessor(table,line,i)

        df = astropy.table.QTable(data=list(map(list,zip(*table))), names=self.headings)
        for name,unit,dtype in zip(self.headings,self.units,self.dtypes):
            df[name].unit = unit
            df[name] = df[name].astype(dtype)
        
        df.meta['footer'] = comment
        df.meta['element'] = self.element
        df.meta['ion'] = self.ion_name
        
        df = self.postprocessor(df)
        
        return df
    
    def preprocessor(self,table,line,index):
        """
        Default preprocessor method run on each line ingested 
        """
        table.append(line)
        
    def postprocessor(self,df):
        """
        Default postprocessor method run on the whole dataframe
        """
        return df
    
    def to_hdf5(self,hf):
        """
        Add datasets to a group for an HDF5 file handler
        """
        pass
        

class Parser(GenericParser,metaclass=ParserFactory):
    def __init__(self,ion_filename):
        super().__init__(ion_filename)

### Preprocessors
Preprocessors are essentially middleware that know the details of the specific filetype we're parsing. This makes it so that there is one common interface to parsing all files and the middleware is selected based on the supplied extension. These are instantiated by the factory and should not be instantiated manually.

In [120]:
class ElvlcParser(GenericParser):
    dtypes = [int,str,int,str,float,float,float]
    units = [None,None,None,None,u.dimensionless_unscaled,1/u.cm,1/u.cm]
    headings = ['level index','configuration','multiplicity',
                'orbital angular momentum','total angular momentum',
                'observed energy','theoretical energy']

    
class FblvlParser(GenericParser):
    dtypes = [int,str,int,int,str,int,float,float]
    units = [None,None,None,None,None,None,1/u.cm,1/u.cm]
    headings = ['level index','configuration','principal quantum number',
                'azimuthal quantum number','orbital angular momentum',
                'multiplicity','observed energy','theoretical energy']

    
class ScupsParser(GenericParser): 
    dtypes = [int,int,float,float,float,int,int,float,'object','object']
    units = [None,None,u.Ry,u.dimensionless_unscaled,1/u.Ry,None,None,
             u.dimensionless_unscaled,u.dimensionless_unscaled,
             u.dimensionless_unscaled]
    headings = ['lower level index','upper level index','delta energy','oscillator strength',
                'high-temperature limit','number of scaled temperatures','Burgess-Tully scaling type',
                'Burgess-Tully scaling parameter','Burgess-Tully scaled temperatures',
                'Burgess-Tully scaled effective collision strengths']
    
    def preprocessor(self,table,line,index):
        if index%3 == 0:
            # main data
            table.append(line)
        else:
            # scaled temperature or collision strengths
            scaled = np.array(line,dtype=float)
            table[-1].append(scaled)
            
class PsplupsParser(GenericParser):
    dtypes = [int,int,int,float,float,float,float]
    units = [None,None,None,u.dimensionless_unscaled,u.Ry,u.dimensionless_unscaled,
             u.dimensionless_unscaled]
    headings = ['lower level index','upper level index','Burgess-Tully scaling type',
                'oscillator strength','delta energy','Burgess-Tully scaling parameter',
                'Burgess-Tully scaled effective collision strengths']
    
    def preprocessor(self,table,line,index):
        line = list(filter(None,('      '.join(line)).split()))
        row = line[:5]
        tmp = line[5]
        i_split = [i for i,char in enumerate(tmp) if char=='-' and tmp[i-1]!='e'][0]
        row += [tmp[:i_split]]
        scups = [tmp[i_split+1:]] 
        tmp_scups = line[6:]
        if len(tmp_scups[-1].split('-')) > 2:
            tmp = tmp_scups[-1]
            i_split = [i for i,char in enumerate(tmp) if char=='-' and tmp[i-1]!='e'][0]
            tmp_scups = tmp_scups[:-1] + [tmp[:i_split],tmp[i_split+1:]]
        scups = np.array(scups+tmp_scups,dtype=float)
        row += [scups]
        table.append(row)
        
            
class EasplomParser(GenericParser):
    dtypes = [int,int,int,float,float,float,float]
    units = [None,None,None,u.dimensionless_unscaled,u.Ry,u.dimensionless_unscaled,
             u.dimensionless_unscaled]
    headings = ['lower level index','upper level index','Burgess-Tully scaling type',
                'oscillator strength','delta energy','Burgess-Tully scaling parameter',
                'Burgess-Tully scaled cross-section']
    
    def preprocessor(self,table,line,index):
        line = list(filter(None,('      '.join(line)).split()))
        scaled_cs = np.array(line[8:],dtype=float)
        row = line[2:8] + [scaled_cs]
        table.append(row)
        
        
class EasplupsParser(EasplomParser):
    dtypes = [int,int,int,float,float,float,float]
    units = [None,None,None,u.dimensionless_unscaled,u.Ry,u.dimensionless_unscaled,
             u.dimensionless_unscaled]
    headings = ['lower level index','upper level index','Burgess-Tully scaling type',
                'oscillator strength','delta energy','upsilon coefficient',
                'excitation-autoionization rate coefficients']
    
    
class WgfaParser(GenericParser):
    dtypes = [np.int,np.int,np.float,np.float,np.float,
              str,np.int,str,np.float,
              str,np.int,str,np.float]
    units = [None,None,u.angstrom,u.dimensionless_unscaled,1/u.s,None,None,None,None,
             None,None,None,None]
    headings = ['lower level index','upper level index',
                'transition wavelength','oscillator strength','radiative decay rate',
                'lower level configuration','lower level multiplicity',
                'lower level orbital angular momentum',
                'lower level total angular momentum',
                'upper level configuration','upper level multiplicity',
                'upper level orbital angular momentum',
                'upper level total angular momentum']
    
    def preprocessor(self,table,line,index):
        ### lower ###
        tmp = line[-2].strip().split()
        del tmp[-1] # delete rogue dash
        tmp_pretty = tmp[-1]
        config = ' '.join(tmp[:-1])
        mult = tmp_pretty[0]
        orb = tmp_pretty[1]
        frac = tmp_pretty[2:]
        if len(frac) == 1:
            frac = frac[0]
        else:
            frac = float(frac.split('/')[0])/float(frac.split('/')[-1])
        lower = [config,mult,orb,frac] 
        ### upper ###
        tmp = line[-1].strip().split()
        tmp_pretty = tmp[-1]
        config = ' '.join(tmp[:-1])
        mult = tmp_pretty[0]
        orb = tmp_pretty[1]
        frac = tmp_pretty[2:]
        if len(frac) == 1:
            frac = frac[0]
        else:
            frac = float(frac.split('/')[0])/float(frac.split('/')[-1])
        upper = [config,mult,orb,frac] 
        ### recombine and assemble ###
        table.append(line[:-2] + lower + upper)
        
        
class CilvlParser(GenericParser):
    dtypes = [int,int,float,float]
    units = [None,None,u.K,(u.cm**3)/u.s]
    headings = ['lower level index','upper level index','temperature',
                'ionization rate coefficient']
    
    def preprocessor(self,table,line,index):
        line = (' '.join(line)).split()
        if index%2 == 0:
            row = line[2:4]
            temperature = 10.**np.array(line[4:],dtype=float)
            row += [temperature]
            table.append(row)
        else:
            rate_coefficient = np.array(line[4:],dtype=float)
            table[-1].append(rate_coefficient)

            
class ReclvlParser(CilvlParser):
    headings = ['lower level index','upper level index','temperature',
                'recombination rate coefficient']
    
    
class RrparamsParser(GenericParser):
    
    def preprocessor(self,table,line,index):
        if index == 0:
            filetype = int(line[0])
            table.append([filetype])
            if filetype == 1:
                self.dtypes = [int,float,float,float,float]
                self.units = [None,(u.cm**3)/u.s,None,u.K,u.K]
                self.headings = ['fit type','A fit parameter','B fit parameter',
                                 'T0 fit parameter','T1 fit parameter']
            elif filetype == 2:
                self.dtypes = [int,float,float,float,float,float,float]
                self.units = [None,(u.cm**3)/u.s,None,u.K,u.K,None,u.K]
                self.headings = ['fit type','A fit parameter','B fit parameter',
                                 'T0 fit parameter','T1 fit parameter','C fit parameter',
                                 'T2 fit parameter']
            elif filetype == 3:
                self.dtypes = [int,float,float]
                self.units = [None,(u.cm**3)/u.s,None]
                self.headings = ['fit type','A rad fit parameter','eta fit parameter']
            else:
                raise ValueError('Unrecognized .rrparams filetype {}'.format(filetype))
        else:
            if table[0][0] == 1 or table[0][0] == 2:
                table[0] += (' '.join(line[3:])).split()
            else:
                table[0] += (' '.join(line[2:])).split()
                
                
class TrparamsParser(GenericParser):
    dtypes = [float,float]
    units = [u.K,(u.cm**3)/u.s]
    headings = ['temperature','total recombination rate']
    
    def preprocessor(self,table,line,index):
        if index > 0:
            super().preprocessor(table,line,index)
            
            
class DrparamsParser(GenericParser):
    
    def preprocessor(self,table,line,index):
        raise NotImplementedError('Parser for .drparams files not yet implemented.')
        
        
class DiparamsParser(GenericParser):
    
    def preprocessor(self,table,line,index):
        raise NotImplementedError('Parser for .diparams files not yet implemented.')

## Trying it out
Parse some example files for all of the filetypes using our parser factory approach.

`.trparams` Files

In [117]:
Parser('fe_2.trparams').parse()

temperature,total recombination rate
K,cm3 / s
float64,float64
10.0,3.48e-11
12.59,3.14e-11
15.85,2.84e-11
19.95,2.58e-11
25.12,2.33e-11
31.62,2.09e-11
39.81,1.87e-11
50.12,1.67e-11


`.rrparams` Files

In [103]:
Parser('al_2.rrparams').parse()

fit type,A fit parameter,B fit parameter,T0 fit parameter,T1 fit parameter,C fit parameter,T2 fit parameter
,cm3 / s,,K,K,,K
int64,float64,float64,float64,float64,float64,float64
2,1.035e-09,0.6535,0.04623,1564000000.0,0.1261,153300.0


In [104]:
Parser('ar_17.rrparams').parse()

fit type,A fit parameter,B fit parameter,T0 fit parameter,T1 fit parameter
,cm3 / s,,K,K
int64,float64,float64,float64,float64
1,2.812e-10,0.5686,10630.0,87790000.0


In [105]:
Parser('co_2.rrparams').parse()

fit type,A rad fit parameter,eta fit parameter
,cm3 / s,
int64,float64,float64
3,1e-12,0.7


`.cilvl` and `.reclvl` Files

In [47]:
Parser('fe_17.cilvl').parse()

lower level index,upper level index,temperature [9],ionization rate coefficient [9]
,,K,cm3 / s
int64,int64,float64,float64
1,2,1122018.4543019629 .. 17782794.100389227,1.05e-16 .. 4.18e-11
1,3,1122018.4543019629 .. 17782794.100389227,6.13e-17 .. 2.5e-11
1,4,1122018.4543019629 .. 17782794.100389227,1.88e-17 .. 8.55e-12
1,5,1122018.4543019629 .. 17782794.100389227,5.27e-17 .. 2.44e-11
1,6,1122018.4543019629 .. 17782794.100389227,1.11e-18 .. 5.95e-13
1,7,1122018.4543019629 .. 17782794.100389227,6.57e-25 .. 1.53e-19
1,8,1122018.4543019629 .. 17782794.100389227,7.06e-25 .. 1.69e-19
1,9,1122018.4543019629 .. 17782794.100389227,6.95e-19 .. 4e-13


In [54]:
Parser('al_11.reclvl').parse()

lower level index,upper level index,temperature [19],recombination rate coefficient [19]
,,K,cm3 / s
int64,int64,float64,float64
1,1,1210.598133550483 .. 1210598133.550483,8.81e-12 .. 7.18e-16
1,2,1210.598133550483 .. 1210598133.550483,8.42e-12 .. 7.77e-17
1,3,1210.598133550483 .. 1210598133.550483,1.68e-11 .. 1.55e-16
1,4,1210.598133550483 .. 1210598133.550483,2.75e-12 .. 2.09e-16
1,5,1210.598133550483 .. 1210598133.550483,3.1e-12 .. 2.79e-17
1,6,1210.598133550483 .. 1210598133.550483,6.17e-12 .. 5.58e-17
1,7,1210.598133550483 .. 1210598133.550483,4.09e-12 .. 8.82e-18
1,8,1210.598133550483 .. 1210598133.550483,6.12e-12 .. 1.32e-17


 `.scups` Files

In [121]:
Parser('fe_6.scups').parse()

lower level index,upper level index,delta energy,oscillator strength,high-temperature limit,number of scaled temperatures,Burgess-Tully scaling type,Burgess-Tully scaling parameter,Burgess-Tully scaled temperatures [15],Burgess-Tully scaled effective collision strengths [15]
,,Ry,,1 / Ry,,,,,
int64,int64,float64,float64,float64,int64,int64,float64,object,object
1,2,0.004657,-1.0,1e-06,15,2,274.5,0.0 .. 1.0,2.08 .. 1e-06
1,3,0.01083,-1.0,1e-06,15,2,118.1,0.0 .. 1.0,0.9837 .. 1e-06
1,4,0.01823,-1.0,1e-06,15,2,70.13,0.0 .. 1.0,0.969 .. 1e-06
1,5,0.1708,-1.0,1e-06,15,2,7.485,0.0 .. 1.0,0.321 .. 1e-06
1,6,0.1726,-1.0,1e-06,15,2,7.407,0.0 .. 1.0,0.39 .. 1e-06
1,7,0.1787,-1.0,1e-06,15,2,7.155,0.0 .. 1.0,0.3522 .. 1e-06
1,8,0.1879,-1.0,1e-06,15,2,6.804,0.0 .. 1.0,0.8292 .. 1e-06
1,9,0.1942,-1.0,1e-06,15,2,6.583,0.0 .. 1.0,0.3891 .. 1e-06


 `.psplups` Files

In [122]:
Parser('ca_17.psplups').parse()

lower level index,upper level index,Burgess-Tully scaling type,oscillator strength,delta energy,Burgess-Tully scaling parameter,Burgess-Tully scaled effective collision strengths [9]
,,,,Ry,,
int64,int64,int64,float64,float64,float64,float64
2,3,2,0.0,0.1021,432.0,3.16e-14 .. 2.418e-13
2,4,2,0.0,0.352,214.5,2.704e-12 .. 5.749e-11
3,4,2,0.0,0.2499,291.5,6.37e-12 .. 4.349e-11


`.elvlc` Files

In [123]:
Parser('h_1.elvlc').parse()

level index,configuration,multiplicity,orbital angular momentum,total angular momentum,observed energy,theoretical energy
,,,,,1 / cm,1 / cm
int64,str3,int64,str1,float64,float64,float64
1,1s,2,S,0.5,0.0,0.0
2,2s,2,S,0.5,82258.956,82303.0
3,2p,2,P,0.5,82258.921,82303.0
4,2p,2,P,1.5,82259.287,82303.0
5,3s,2,S,0.5,97492.224,97544.0
6,3p,2,P,0.5,97492.213,97544.0
7,3p,2,P,1.5,97492.321,97544.0
8,3d,2,D,1.5,97492.321,97544.0


 `.easplom` Files

In [124]:
Parser('fe_6.easplom').parse()

lower level index,upper level index,Burgess-Tully scaling type,oscillator strength,delta energy,Burgess-Tully scaling parameter,Burgess-Tully scaled cross-section [5]
,,,,Ry,,
int64,int64,int64,float64,float64,float64,float64
1,2,2,0.0,7.633,1.2,0.1138 .. 0.2265
1,3,4,0.3533,7.86,1.3,1.225 .. 0.04335
1,4,4,0.03593,9.028,1.6,0.1132 .. 0.004365
1,5,4,7.989e-06,10.41,1.8,0.01005 .. 3.055e-05
1,6,4,0.2973,9.017,1.8,0.2545 .. 0.03355
1,7,4,0.1625,10.49,1.6,0.08102 .. 0.01549
1,8,4,0.2093,9.777,1.6,0.1767 .. 0.02225
1,9,1,0.1796,11.41,1.6,0.01348 .. 0.01571


 `.easplups` Files

In [125]:
Parser('fe_6.easplups').parse()

lower level index,upper level index,Burgess-Tully scaling type,oscillator strength,delta energy,upsilon coefficient,excitation-autoionization rate coefficients [5]
,,,,Ry,,
int64,int64,int64,float64,float64,float64,float64
1,2,2,0.0,7.633,1.8,0.1138 .. 0.2265
1,3,4,0.3533,7.86,1.3,1.225 .. 0.04335
1,4,4,0.03593,9.028,2.0,0.1132 .. 0.004365
1,5,4,7.989e-06,10.41,2.4,0.01005 .. 3.055e-05
1,6,4,0.2973,9.017,2.0,0.2545 .. 0.03355
1,7,4,0.1625,10.49,1.6,0.08102 .. 0.01549
1,8,4,0.2093,9.777,1.7,0.1767 .. 0.02225
1,9,1,0.1796,11.41,2.2,0.01348 .. 0.01571


`.fblvl` Files

In [126]:
Parser('ar_16.fblvl').parse()

level index,configuration,principal quantum number,azimuthal quantum number,orbital angular momentum,multiplicity,observed energy,theoretical energy
,,,,,,1 / cm,1 / cm
int64,str6,int64,int64,str1,int64,float64,float64
1,1s22s,2,0,s,2,0.0,0.0
2,1s22p,2,1,p,6,274026.667,274039.521
3,1s23s,3,0,s,2,4176024.0,4177011.75
4,1s23p,3,1,p,6,4244333.333,4253472.333
5,1s23d,3,2,d,10,4282560.0,4284798.4
6,1s24s,4,0,s,2,5605760.0,5606934.0
7,1s24p,4,1,p,6,5636873.333,5638415.833
8,1s24d,4,2,d,10,5649608.0,5651632.8


`.wgfa` Files

In [127]:
Parser('he_2.wgfa').parse()

lower level index,upper level index,transition wavelength,oscillator strength,radiative decay rate,lower level configuration,lower level multiplicity,lower level orbital angular momentum,lower level total angular momentum,upper level configuration,upper level multiplicity,upper level orbital angular momentum,upper level total angular momentum
,,Angstrom,,1 / s,,,,,,,,
int64,int64,float64,float64,float64,str2,int64,str1,float64,str2,int64,str1,float64
1,2,0.0,0.0,526.6,1s,2,S,0.5,2s,2,S,0.5
1,3,303.786,0.2772,10020000000.0,1s,2,S,0.5,2p,2,P,0.5
1,4,303.781,0.5552,10030000000.0,1s,2,S,0.5,2p,2,P,1.5
1,6,256.318,0.05268,2675000000.0,1s,2,S,0.5,3p,2,P,0.5
1,7,256.317,0.1055,2679000000.0,1s,2,S,0.5,3p,2,P,1.5
1,11,243.027,0.01931,1090000000.0,1s,2,S,0.5,4p,2,P,0.5
1,12,243.027,0.03867,1092000000.0,1s,2,S,0.5,4p,2,P,1.5
1,18,237.331,0.009284,549800000.0,1s,2,S,0.5,5p,2,P,0.5


## Check Raw Text
Call tail on the actual file to make sure we are parsing the right stuff.

In [108]:
%%bash
head -n 64 $XUVTOP/fe/fe_2/fe_2.trparams

   62
  1.0000e+01  3.4800e-11
  1.2590e+01  3.1400e-11
  1.5850e+01  2.8400e-11
  1.9950e+01  2.5800e-11
  2.5120e+01  2.3300e-11
  3.1620e+01  2.0900e-11
  3.9810e+01  1.8700e-11
  5.0120e+01  1.6700e-11
  6.3100e+01  1.4800e-11
  7.9430e+01  1.3100e-11
  1.0000e+02  1.1600e-11
  1.2590e+02  1.0200e-11
  1.5850e+02  9.0200e-12
  1.9950e+02  7.9700e-12
  2.5120e+02  7.0400e-12
  3.1620e+02  6.2700e-12
  3.9810e+02  5.6500e-12
  5.0120e+02  5.1500e-12
  6.3100e+02  4.7300e-12
  7.9430e+02  4.3400e-12
  1.0000e+03  3.9500e-12
  1.2590e+03  3.5500e-12
  1.5850e+03  3.1400e-12
  1.9950e+03  2.7500e-12
  2.5120e+03  2.3700e-12
  3.1620e+03  2.0200e-12
  3.9810e+03  1.7200e-12
  5.0120e+03  1.4700e-12
  6.3100e+03  1.3500e-12
  7.9430e+03  1.5200e-12
  1.0000e+04  2.3000e-12
  1.2590e+04  3.8900e-12
  1.5850e+04  6.1900e-12
  1.9950e+04  8.7200e-12
  2.5120e+04  1.0900e-11
  3.1620e+04  1.2100e-11
  3.9810e+04  1.2400e-11
  5.0120e+04  1.1700e-11
  6.3100e+04  1.0500e-11
  7.9430e+04  8.950

In [100]:
%%bash
head -n 10 $XUVTOP/co/co_2/co_2.rrparams

    3
   27    2  1.0000e-12  7.0000e-01
-1
%file:  co_2.rrparams
% radiative recombination parameters, interpolated from:
  Shull, J. M., Van Steenberg, M., 1982, ApJSS, 48, 95
  Landini, M., Monsignori Fossi, B. C., 1990, A\&ASS, 82, 229
  Landini, M., Monsignori Fossi, B. C., 1991, A\&ASS, 91, 183
  Mazzitelli, G., Mattioli, M., 2002, ADNDT, 82, 313
 produced for the CHIANTI atomic database for astrophysical spectroscopy


In [93]:
%%bash
head -n 10 $XUVTOP/ar/ar_17/ar_17.rrparams

    1
   18   17    1  2.8120e-10   0.56860  1.0630e+04  8.7790e+07
-1
%file:  ar_17.rrparams
 Radiative recombination coefficients
Badnell, N. R., 2006, ApJS, 167, 334
http://adsabs.harvard.edu/abs/2006ApJS..167..334B
Radiative Recombination Data for Modeling Dynamic Finite-Density Plasmas
 produced for the CHIANTI atomic database for astrophysical spectroscopy
 created by Ken Dere (GMU):  Wed Feb 15 11:06:15 2012


In [74]:
%%bash
head -n 10 $XUVTOP/al/al_2/al_2.rrparams

    2
   13    2    1  1.0350e-09   0.65350 4.6230e-02 1.5640e+09   0.12610  1.5330e+05
-1
%file:  al_2.rrparams
 Radiative recombination coefficients
Altun, Z., Yumak, I., Badnell, N.R., Loch, S.D., Pindzola, M.D.
2007, A&A, 474, 1051
http://cdsads.u-strasbg.fr/cgi-bin/bib_query?2007A%26A...474.1051A
Dielectronic recombination data for dynamic finite-density plasmas. XIII. The magnesium isoelectronic sequence
 produced for the CHIANTI atomic database for astrophysical spectroscopy


In [64]:
%%bash
head -n 10 $XUVTOP/ca/ca_17/ca_17.drparams

    1
   20   17  1.9900e+04  8.6240e+04  3.1800e+05  8.4800e+05  5.7350e+06  3.3000e+07  7.2680e+07  0.0000e+00  0.0000e+00
   20   17  2.8920e-03  7.6630e-03  2.6890e-02  5.7180e-02  2.6810e-01  1.7090e-01  2.2620e-02  0.0000e+00  0.0000e+00
-1
%file:  ca_17.drparams
%parameters for dielectronic recombination rate coefficients
Colgan, J., Pindzola, M.S., Whiteford, A.D., Badnell, N.R., 2003, A&A, 412, 597
Dielectronic recombination data for dynamic finite-density plasmas. III. The beryllium isoelectronic sequence
http://cdsads.u-strasbg.fr/cgi-bin/bib_query?2003A%26A...412..597C
 produced for the CHIANTI atomic database for astrophysical spectroscopy


In [4]:
%%bash
head -n 10 $XUVTOP/fe/fe_17/fe_17.cilvl

 26 17  1   2      6.05      6.20      6.35      6.50      6.65      6.80      6.95      7.10      7.25
 26 17  1   2 1.050e-16 4.830e-15 7.620e-14 5.610e-13 2.400e-12 6.980e-12 1.530e-11 2.730e-11 4.180e-11
 26 17  1   3      6.05      6.20      6.35      6.50      6.65      6.80      6.95      7.10      7.25
 26 17  1   3 6.130e-17 2.850e-15 4.510e-14 3.330e-13 1.430e-12 4.160e-12 9.120e-12 1.630e-11 2.500e-11
 26 17  1   4      6.05      6.20      6.35      6.50      6.65      6.80      6.95      7.10      7.25
 26 17  1   4 1.880e-17 9.000e-16 1.460e-14 1.100e-13 4.770e-13 1.400e-12 3.090e-12 5.550e-12 8.550e-12
 26 17  1   5      6.05      6.20      6.35      6.50      6.65      6.80      6.95      7.10      7.25
 26 17  1   5 5.270e-17 2.540e-15 4.140e-14 3.120e-13 1.360e-12 3.980e-12 8.810e-12 1.580e-11 2.440e-11
 26 17  1   6      6.05      6.20      6.35      6.50      6.65      6.80      6.95      7.10      7.25
 26 17  1   6 1.110e-18 5.620e-17 9.450e-16 7.280e-15 3.220e-14 

In [ ]:
%%bash
head -n 10 $XUVTOP/fe/fe_6/fe_6.scups

In [ ]:
%%bash
head -n 10 $XUVTOP/ca/ca_17/ca_17.psplups

In [ ]:
%%bash
head -n 10 $XUVTOP/h/h_1/h_1.elvlc

In [ ]:
%%bash
head -n 100 $XUVTOP/fe/fe_6/fe_6.easplom

In [ ]:
%%bash
head -n 10 $XUVTOP/fe/fe_6/fe_6.easplups

In [ ]:
%%bash
head -n 30 $XUVTOP/h/h_1/h_1.fblvl

In [ ]:
%%bash
head -n 100 $XUVTOP/fe/fe_3/fe_3.wgfa